# License
    IPython notebook for simulating the linear wave equation with CUDA
    Copyright (C) 2015, 2018 Andre.Brodtkorb@ifi.uio.no

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
%matplotlib inline 

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import pycuda.compiler as cuda_compiler
import pycuda.driver as cuda_driver
from pycuda.gpuarray import GPUArray

import IPythonMagic
from Timer import Timer

In [2]:
%setup_logging
%cuda_context_handler context

Python version 3.6.5 |Anaconda custom (64-bit)| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
Registering context in user workspace
Creating context
PyCUDA version 2017.1.1
CUDA version (9, 1, 0)
Driver version 9010
Using 'GeForce 840M' GPU
 => compute capability: (5, 0)
 => memory: 1947 / 2048 MB available
Created context handle <875615065328>
Using CUDA cache dir c:\Users\anbro\Documents\projects\demos\MilanoGPU2018\notebooks\cuda_cache


# Linear Wave Equation in 2D
The acoustic wave equation in 2D can be written
$$
\begin{align}
\frac{\partial^2 u}{\partial t^2} &= c^2 \nabla^2 u\\
&= c^2 \left[ \frac{\partial^2 u}{\partial x^2} +  \frac{\partial^2 u}{\partial y^2} \right]
\end{align}
$$
where $u$ is the string position, and c is a material specific constant. 

By approximating the temporal derivative with a backward difference, and the spatial derivative with a central difference, we get
$$
\frac{1}{\Delta t^2} (u_{i, j}^{n+1} - 2u_{i, j}^{n} + u_{i, j}^{n-1}) 
= c \left [
\frac{1}{\Delta x^2}(u_{i-1, j}^n - 2u_{i, j}^n + u_{i+1, j}^n)
+ \frac{1}{\Delta y^2}(u_{i, j-1}^n - 2u_{i, j}^n + u_{i, j+1}^n)
\right]
$$
and gathering $u^n+1$ on the left hand side and $u^n$on the right, we write
$$
u^{n+1}_{i,j} = 2u_{i,j}^n - u_{i, j}^{n-1}
+ \frac{\kappa\Delta t^2}{\Delta x^2}(u_{i-1, j}^n - 2u_{i, j}^n + u_{i+1, j}^n)
+ \frac{\kappa\Delta t^2}{\Delta y^2}(u_{i, j-1}^n - 2u_{i, j}^n + u_{i, j+1}^n)
$$
This discretization is unstable if the following CFL condition is not met
$$
\frac{1}{2} \gt \frac{\kappa\Delta t}{\Delta x^2}, \qquad
\frac{1}{2} \gt \frac{\kappa\Delta t}{\Delta y^2}
$$
or 
$$
\Delta t \lt \text{min}\left(\frac{\Delta x^2}{2\kappa}, \frac{\Delta y^2}{2\kappa}\right)
$$

In [3]:
class LinearWaveGPU(object):
    def __init__(self):
        pass
        
    def initialize(self, u1, u0, kappa, dx, dy, dt, block_width=16, block_height=4):
        self.block_size = (block_width, block_height, 1)
        self.grid_size = (int(np.ceil(nx / block_width)), int(np.ceil(ny / block_height)), 1)
        self.stream = cuda_driver.Stream()
        
        self.u2_g = GPUArray(u1.shape, u1.dtype)
        self.u1_g = GPUArray(u1.shape, u1.dtype)
        self.u0_g = GPUArray(u0.shape, u0.dtype)
        
        self.u0_g.set_async(u0, stream=self.stream)
        self.u1_g.set_async(u1, stream=self.stream)
        
        self.kappa = np.float32(kappa)
        self.dx = np.float32(dx)
        self.dy = np.float32(dy)
        self.dt = np.float32(dt)
        self.nx = np.float32(u0.shape[1])
        self.ny = np.float32(u0.shape[0])
        
        
        cuda_kernel = \
        """
        #define BLOCK_WIDTH {:d}
        #define BLOCK_HEIGHT {:d}
        """.format(block_width, block_height) \
        + \
        """
        __global__ void linearWaveEqn(float* u2, const float* u1, const float* u0, float kappa, float dx, float dy, float dt, int nx, int ny) {
            //Plus one to skip the "ghost cells"
            int i = blockIdx.x*blockDim.x + threadIdx.x + 1;
            int j = blockIdx.y*blockDim.y + threadIdx.y + 1;
            int tx = threadIdx.x + 1;
            int ty = threadIdx.y + 1;

            //First read into shared memory, including the local ghost cells / apron
            __shared__ float u1_shared[BLOCK_HEIGHT+2][BLOCK_WIDTH+2];
            for (int l=threadIdx.y; l<BLOCK_HEIGHT+2; l += blockDim.y) {
                int t = min(blockIdx.y*blockDim.y + l, ny);
                for (int k=threadIdx.x; k<BLOCK_WIDTH+2; k += blockDim.x) {
                    int s = min(blockIdx.x*blockDim.x + k, nx);
                    u1_shared[l][k] = u1[t*nx + s];
                }
            }
            __syncthreads();

            //Then fix the boundary conditions
            if (i == 1) {
                u1_shared[ty][tx-1] = u1_shared[ty][tx];
            }
            else if (i == nx) {
                u1_shared[ty][tx+1] = u1_shared[ty][tx];
            }
            else if (j == 1) {
                u1_shared[ty-1][tx] = u1_shared[ty][tx];
            }
            else if (j == ny) {
                u1_shared[ty+1][tx] = u1_shared[ty][tx];
            }
            __syncthreads();

            if (i > 0 && i < nx && j > 0 && j < ny) {
                u2[j*nx + i] = 2.0f*u1_shared[ty][tx] - u0[j*nx+i]
                    + kappa*dt/(dx*dx) * (u1_shared[ty][tx-1] - 2*u1_shared[ty][tx] + u1_shared[ty][tx+1])
                    + kappa*dt/(dy*dy) * (u1_shared[ty-1][tx] - 2*u1_shared[ty][tx] + u1_shared[ty+1][tx]);
            }
        }
        """
        self.module = cuda_compiler.SourceModule(cuda_kernel, \
                                            options=['--use_fast_math'])
        self.linear_wave_kernel = self.module.get_function("linearWaveEqn");
        self.linear_wave_kernel.prepare("PPPffffii")
        
        
        
    def step(self):
        self.linear_wave_kernel.prepared_async_call(self.grid_size, self.block_size, self.stream, \
                                                    self.u2_g.gpudata, self.u1_g.gpudata, self.u0_g.gpudata, \
                                                    kappa, dx, dy, dt, nx, ny)
        self.u0_g, self.u1_g, self.u2_g = self.u1_g, self.u2_g, self.u0_g
    
    
    def download(self):
        #self.u2 = cuda_driver.pagelocked_empty(u1.shape, u1.dtype)
        u1 = np.empty(self.u1_g.shape, self.u1_g.dtype)
        self.u1_g.get(u1)
        return u1
        
        
        
        
        
#Create test input data
kappa = 1.0
nx, ny = 100, 50
dx = 1.0
dy = 2.0
dt = 0.2 * min(dx / (2.0*kappa), dy / (2.0*kappa))

u0 = np.zeros((ny, nx), dtype=np.float32)
for j in range(ny):
    for i in range(nx):
        x = (i - nx/2.0) * dx
        y = (j - ny/2.0) * dy
        if (np.sqrt(x**2 + y**2) < 10*min(dx, dy)):
            u0[j, i] = 10.0
u1 = np.copy(u0)

with Timer("Initialization") as t:
    simulator = LinearWaveGPU()
    simulator.initialize(u1, u0, kappa, dx, dy, dt)

Initialization: 483.888149 ms


In [4]:
fig = plt.figure(figsize=(12, 8))
domain_extent = [0, nx*dx, 0, ny*dy]
im = plt.imshow(u1, origin='bottom', interpolation='none', vmin=-5, vmax=5, extent=domain_extent)
plt.colorbar()

def animate(i):
    if (i > 0):
        for k in range(10):
            simulator.step()
    u2 = simulator.download()
    im.set_data(u2)
    

plt.rcParams["animation.html"] = "jshtml"
anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
plt.close()
anim

Animation.save using <class 'matplotlib.animation.HTMLWriter'>
frame size in pixels is 864 x 576
findfont: Matching :family=sans-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0 to DejaVu Sans ('C:\\Users\\anbro\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\DejaVuSans.ttf') with score of 0.050000
FileMovieWriter.frame_sink: saving frame 0 to fname=None0000000.png
MovieWriter -- Command stdout:

MovieWriter -- Command stderr:

